In [ ]:
!pip install redis
!pip install python-dotenv
!pip install slugify
!pip install pydantic
!pip install tqdm

In [35]:
%env REDIS_HOST=127.0.0.1
%env REDIS_PORT=6379
%env REDIS_PASSWORD=

env: REDIS_HOST=127.0.0.1
env: REDIS_PORT=6379
env: REDIS_PASSWORD=


In [49]:
import os

from dotenv import load_dotenv
from redis import Redis
from redis.commands.search.field import NumericField, TextField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.suggestion import Suggestion


In [37]:
load_dotenv()

redis_client = Redis(
    host=os.environ["REDIS_HOST"],
    port=int(os.environ["REDIS_PORT"]),
    password=os.environ.get("REDIS_PASSWORD"),
)

os.environ["REDIS_HOST"]

'127.0.0.1'

# Initialize the index

In [51]:
schema = (
    TextField("$.title", as_name="title", weight=10.0),
    TextField("$.description", as_name="description"),
    NumericField("$.date", as_name="date"),
    TextField("$.category", as_name="category"),
)

redis_client.ft("articles").create_index(
    schema, definition=IndexDefinition(prefix=["articles:"], index_type=IndexType.JSON)
)

b'OK'

In [48]:
redis_client.ft("articles").dropindex()

b'OK'

# Import Data

In [40]:
import datetime
import json
import os
from typing import Union
from redis.commands.json.path import Path
from pydantic import BaseModel
from slugify import slugify
from tqdm import tqdm

In [41]:

class Article(BaseModel):
    id: str
    title: str
    url: str
    category: str
    description: str
    date: Union[str, datetime.datetime, int]
    authors: str

    class Config:
        json_encoders = {datetime.datetime: lambda v: int(v.timestamp())}

DATA_PATH = "../data/News_Category_Dataset_v3.json"


In [52]:
with tqdm(total=os.path.getsize(DATA_PATH)) as pbar:
    with open(DATA_PATH, "r") as f:
        for line in f:
            pbar.update(len(line))
            data = json.loads(line)
            id = slugify(data["title"])
            try:
                data["id"] = id
                data["date"] = datetime.datetime.strptime(data["date"], "%Y-%m-%d")
                article = Article(**data)
                article_dict = json.loads(article.json())
                redis_client.json().set(f"articles:{article.id}", Path.root_path(), article_dict)
                # Add autocomplete suggestion
                result = redis_client.ft("articles").sugadd(
                    "articles", Suggestion(string=article.title, score=article_dict["date"])
                )
            except Exception as e:
                continue

  0%|          | 0/7830349 [00:00<?, ?it/s]/var/folders/7c/tthb1grn2ynbpnrs4ht8jqnr0000gn/T/ipykernel_29579/2725058047.py:11: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.3/migration/
  article_dict = json.loads(article.json())
100%|██████████| 7830349/7830349 [00:15<00:00, 510952.82it/s]
